In [118]:
import os
import numpy as np
import torch

device = 'cpu'
block_size = 1023
batch_size = 2
dataset = ''

# poor man's data loader
data_dir = dataset
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint8, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint8, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # Ensure the starting index is a multiple of block_size
    ix = torch.randint(0, len(data) // (block_size + 1), (batch_size,)) * (block_size + 1)
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [119]:
print(len(train_data))
print(len(val_data) / 1024)

5162455040
102887.0


In [120]:
import pickle
import os


# Usage
# meta_file_path = os.path.join(os.path.dirname(__file__), 'meta.pkl')
file_path = 'meta.pkl'
with open(file_path, 'rb') as f:
    data = pickle.load(f)
print(data)

itos = data['itos']

def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

{'vocab_size': 32, 'itos': {0: ' ', 1: '#', 2: '+', 3: '-', 4: '.', 5: '0', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: ';', 16: '=', 17: 'B', 18: 'K', 19: 'N', 20: 'O', 21: 'Q', 22: 'R', 23: 'a', 24: 'b', 25: 'c', 26: 'd', 27: 'e', 28: 'f', 29: 'g', 30: 'h', 31: 'x'}, 'stoi': {' ': 0, '#': 1, '+': 2, '-': 3, '.': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, ';': 15, '=': 16, 'B': 17, 'K': 18, 'N': 19, 'O': 20, 'Q': 21, 'R': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'x': 31}}


In [141]:
X, Y = get_batch('val')
print("Batch")
print(X)
print("y")
print(Y)

Batch
tensor([[15,  6,  4,  ..., 14,  4, 30],
        [15,  6,  4,  ..., 21, 31, 26]])
y
tensor([[ 6,  4, 27,  ...,  4, 30, 31],
        [ 6,  4, 27,  ..., 31, 26, 13]])


In [122]:
print(decode(X[0].tolist()))
print(decode(X[1].tolist()))
print(decode(Y[0].tolist()))
print(decode(Y[1].tolist()))

;1.c4 e5 2.Nc3 Nf6 3.Nf3 Nc6 4.e3 Bb4 5.Be2 e4 6.Nd4 Bxc3 7.bxc3 Ne7 8.Ba3 d6 9.c5 dxc5 10.Bxc5 O-O 11.h3 b6 12.Bxe7 Qxe7 13.O-O c5 14.Nb3 Qc7 15.a4 a5 16.d4 exd3 17.Qxd3 Ba6 18.c4 Rfe8 19.Bf3 Bb7 20.Bxb7 Qxb7 21.Rfd1 Qe7 22.Qd6 Qe4 23.Nd2 Qe6 24.Qxe6 Rxe6 25.Kf1 Rd8 26.g4 Red6 27.Ke2 g5 28.f3 Kg7 29.Nf1 h6 30.Rxd6 Rxd6 31.Ng3 Kh7 32.Nf5 Rc6 33.Rd1 Ne8 34.Rd8 Re6 35.e4 Nf6 36.Rf8 Ne8 37.Rxf7+ Kg8 38.Rb7 Kf8 39.Kd3 Rf6 40.f4 gxf4 41.e5 Re6 42.Ke4 h5 43.Kxf4 hxg4 44.hxg4 Rc6 45.Ne7 Rh6 46.Nd5 Rg6 47.g5 Ng7 48.Nxb6 Ke8 49.Nd5 Kf8 50.Ra7 Re6 51.Rxa5 Re8 52.Ra7 Ne6+ 53.Kg4 Nxg5 54.Nf6 Rxe5 55.Nd7+ Ke8 56.Nxe5 Ne6 57.a5 Kd8 58.Rd7+ Kc8 59.a6 Nc7 60.a7 Na8 61.Kg3 Nb6 62.Rf7 Kd8 63.a8=Q+ Nxa8 64.Rf8+ Kc7 65.Rxa8 Kb7 66.Ra1 Kb8 67.Ra6 Kc7 68.Nd3 Kd7 69.Nxc5+ Kc7 70.Kf4 Kb8 71.Ke5 Kc7 72.Ke6 Kc8 73.Kd6 Kd8 74.Ra8#;1.e4 c5 2.Nf3 Nc6 3.Bb5 e6 4.O-O Nge7 5.Re1 b6 6.d4 cxd4 7.Nxd4 e5 8.Nf3 a6 9.Bc4 Bb7 10.Ng5 d5 11.Bxd5 Nxd5 12.exd5 Nb4 13.Rxe5+ Qe7 14.Rxe7+ Kd8 15.Nxf7+ Kxe7 16.Qe1+ Kd7 17.Qe6+ Kc7